In [ ]:
!pip install qiskit
!pip install qiskit-aer
!pip install pylatexenc

In [ ]:
# utilities
import matplotlib.pyplot as plt
import numpy as np
from math import floor, sqrt
from copy import deepcopy

# importing Qiskit
from qiskit_aer import Aer
from qiskit import transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

# import basic plot tools
from qiskit.visualization import plot_histogram

In [ ]:
# Qubits required to hold each value in each tile
VALUE_MAG = 4

In [ ]:

# class to represent squares on the sudoku board
class Square(QuantumRegister):

    # row and column are 0 indexed 0-8
    # value is 0 for null, and 1-9 for numbers
    def __init__(self, value: int, row: int, col: int):

        # init register
        super().__init__(VALUE_MAG, name=f"({row},{col})")

        # hold attributes
        self.value = value

        # create 0 padded bitstring
        self.bitstring = str(bin(value))[2:].zfill(VALUE_MAG)

        # set position
        self.coords = (row, col)
        self.row = row
        self.col = col

        # box is 0 indexed 0-8 from top left to bottom right
        self.box = (row // 3) * 3 + (col // 3)

    def add_to_circuit(self, circuit: QuantumCircuit):

        circuit.add_register(self)

        # set bitstring values
        for i in range(VALUE_MAG):

            if self.bitstring[i] == '1':
            
                # bit flip if 1
                circuit.x(self[i])

        return circuit


In [ ]:
# think about adding more quantum registers to preserve original input without uncomputing

# defining a function to check if 2 squares are equal
def nequal(qc: QuantumCircuit, registerA: QuantumRegister, registerB: QuantumRegister, output: QuantumRegister):
    qc.cx(registerA[0], registerB[0])
    qc.cx(registerA[1], registerB[1])
    qc.cx(registerA[2], registerB[2])
    qc.cx(registerA[3], registerB[3])
    qc.x(registerB[0])
    qc.x(registerB[1])
    qc.x(registerB[2])
    qc.x(registerB[3])
    qc.mcx([registerB[0], registerB[1], registerB[2], registerB[3]], output)
    qc.x(output)

# reverts values of compared qubits
def uncompute_nequal(qc: QuantumCircuit, registerA: QuantumRegister, registerB: QuantumRegister):
    qc.cx(registerA[0], registerB[0])
    qc.cx(registerA[1], registerB[1])
    qc.cx(registerA[2], registerB[2])
    qc.cx(registerA[3], registerB[3])
    qc.x(registerB[0])
    qc.x(registerB[1])
    qc.x(registerB[2])
    qc.x(registerB[3])

def set_initials(qc, int0: QuantumRegister, int1: QuantumRegister, int0str: str, int1str: str):

    for idx, i in enumerate(int0str):
        if i == "1":
            qc.x(int0[idx])

    for idx, i in enumerate(int1str):
        if i == "1":
            qc.x(int1[idx])



Here is demonstrated the nequal function with int0 and int1 that are both integers represented by the bit strings being set

In [ ]:
int0 = QuantumRegister(4, name="int0")
int1 = QuantumRegister(4, name="int1")
condition = QuantumRegister(1, name="c")
output = ClassicalRegister(1, name="output")

qc = QuantumCircuit(int0, int1, condition, output)

# add initial values
set_initials(qc, int0, int1, "1100", "1000")
qc.barrier()

# setup equal gate
nequal(qc, int0, int1, condition)
qc.barrier()
uncompute_nequal(qc, int0, int1)
qc.measure(condition, output[0])


qc.draw('mpl', fold=-1)


In [ ]:
# Simulate and plot results
aer_simulator = Aer.get_backend('aer_simulator')
transpiled_qc = transpile(qc, aer_simulator)
result = aer_simulator.run(transpiled_qc).result()
plot_histogram(result.get_counts())

Here are the results from the nequal function

Let's define the data structure that will hold information about the sudoku board, and some functions to display the board.

In [ ]:
class Board:

    def __init__(self, value_array: list[list[int]], rows: int = 9, cols: int = 9, boxes: int = 9, unknown_indicator: int = 0):

        # board shape
        self.rows = rows
        self.cols = cols
        self.boxes = boxes

        # values
        self.values = value_array

        # bit string for each integer passed
        self.bit_strings = [[str(bin(i))[2:].zfill(VALUE_MAG) for i in row] for row in self.values]

        # bool array that's true if the value is a 0 or unknown value to be put in superposition
        self.unknown = [[val == unknown_indicator for val in row] for row in self.values]

        # map of how to init every qubit, weather with h (unknown int), x (1 bit in known int), or nothing (0 bit in known bit)
        # holds strings with either "x", "h", or "nothing"
        self.circuit_map = []

        # map of bools that indicate qubits that get passed through the diffuser
        self.diffuser_map = []

        # iterate through spaces on board
        for row_num in range(self.rows):
            for col_num in range(self.cols):

                # check if unknown square
                if self.unknown[row_num][col_num]:

                    # add all h gates for the value's bits
                    self.circuit_map.extend(["h" for i in range(VALUE_MAG)])
                    self.diffuser_map.extend([True for i in range(VALUE_MAG)])
                
                else:

                    # add bits
                    for bit in self.bit_strings[row_num][col_num]:
                        self.circuit_map.append("x" if int(bit) else "nothing")
                    self.diffuser_map.extend([False for i in range(VALUE_MAG)])
        
        self.diffuser_map = np.array(self.diffuser_map)


    def add_values(self, qc: QuantumCircuit):

        for idx, i in enumerate(self.circuit_map):

            # if hadamard
            if i == "h":
                qc.h(idx)

            # if x
            elif i == "x":
                qc.x(idx)


# takes in 2D array or 0-9 values and displays 3x3 board of numbers
# code generated by ChatGPT
def display_3by3(board):
    fig, ax = plt.subplots(figsize=(3, 3))
    ax.set_xlim(0, 3)
    ax.set_ylim(0, 3)
    ax.set_xticks(np.arange(0, 3, 1))
    ax.set_yticks(np.arange(0, 3, 1))

    # Draw grid
    ax.grid(True, linewidth=2)
    ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

    # Fill numbers
    for i in range(3):
        for j in range(3):
            num = board[i][j]
            if num != 0:
                ax.text(j + 0.5, 2.5 - i, str(num), fontsize=20,
                        ha='center', va='center')

    plt.show()

# takes in 2D array or 0-9 values and displays 9x9 board of numbers
# code generated by ChatGPT
def display_9by9(board):

    fig, ax = plt.subplots(figsize=(5, 5))
    ax.set_xlim(0, 9)
    ax.set_ylim(0, 9)
    ax.set_xticks(np.arange(0, 9, 1))
    ax.set_yticks(np.arange(0, 9, 1))
    ax.set_xticks(np.arange(0, 9, 3), minor=True)
    ax.set_yticks(np.arange(0, 9, 3), minor=True)

    # Draw grid
    ax.grid(True, which='minor', linewidth=2)
    ax.grid(True, which='major', linewidth=0.5, linestyle='--')

    # Hide axis labels
    ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

    # Fill numbers
    for i in range(9):
        for j in range(9):
            num = board[i][j]
            if num != 0:
                ax.text(j + 0.5, 8.5 - i, str(num), fontsize=16,
                        ha='center', va='center')

    plt.show()


Lets define the oracle function

In [ ]:
def oracle(qc: QuantumCircuit,
           squares: list[Square], 
           square_rows: dict[dict[str, Square]], 
           square_cols: dict[dict[str, Square]], 
           square_boxes: dict[dict[str, Square]],
           conditions: QuantumRegister,
           auxiliary: QuantumRegister
           ):
    
    # ADD BINARY INTEGER CLIPPERS
    # ADD NON-0

    # COMPUTE

    # copy square organizers
    square_rows_compute = deepcopy(square_rows)
    square_cols_compute = deepcopy(square_cols)
    square_boxes_compute = deepcopy(square_boxes)

    # nequal conditions record for uncomputing
    nequal_pairs = []

    # add conditions
    condition_idx = 0
    for square in squares:

        # create set of other squares to compare to
        compare_to = set()

        # remove current square from col row and box
        square_rows_compute[square.row].pop(square.name)
        square_cols_compute[square.col].pop(square.name)
        square_boxes_compute[square.box].pop(square.name)

        # add other squares in box
        compare_to.update(set(square_boxes_compute[square.box].values()))

        # add other squares in row
        compare_to.update(set(square_rows_compute[square.row].values()))

        # add other squares in col
        compare_to.update(set(square_cols_compute[square.col].values()))

        # create conditions
        for other_square in compare_to:
            nequal(qc, square, other_square, conditions[condition_idx])
            uncompute_nequal(qc, square, other_square)
            nequal_pairs.append((square, other_square))

            condition_idx += 1

    # set final condition
    qc.mcx(conditions, auxiliary)

    # UNCOMPUTE

    # undo conditions
    condition_idx = 0
    for square, other_square in nequal_pairs:
        
        nequal(qc, square, other_square, conditions[condition_idx])
        uncompute_nequal(qc, square, other_square)

        condition_idx += 1


NOTE:
    All integers will read from top to bottom, left to right. In other words on the circuit diagram, the lowest qubit is $2^0$ and the highest qubit is $2^3$. As bitstrings, the values all read like normal numbers, intuitively sort of backwards though.

Next lets define the diffuser

In [ ]:
def diffuser(qc, qubits):

    # h gates
    for qubit in qubits:
        qc.h(qubit)
    
    # x gates
    for qubit in qubits:
        qc.x(qubit)
    
    # mcz
    qc.h(len(qubits) - 1)
    qc.mcx(list(range(len(qubits) - 1)), len(qubits) - 1)
    qc.h(len(qubits) - 1)

    # undo x gates
    for qubit in qubits:
        qc.x(qubit)
  
    # undo h gates
    for qubit in qubits:
        qc.h(qubit)

Now to create a small sample circuit with just one box

In [ ]:
# circuit constants
ROW_COUNT = 3
COL_COUNT = 3
BOX_COUNT = 1
CONDITION_COUNT = 36

# starting values for board
starting_values = [[1, 0, 3], [4, 0, 6], [7, 0, 9]]

# number of oracle-diffuser iterations for search
unknown_squares = sum([len([i for i in j if i != 0]) for j in starting_values])
CIRCUIT_ITERATIONS = floor(sqrt(9 ** unknown_squares)) + 1

# create circuit
qc = QuantumCircuit()

# squares stored by relations
squares = []
square_rows = [dict() for i in range(ROW_COUNT)]
square_cols = [dict() for i in range(COL_COUNT)]
square_boxes = [dict() for i in range(BOX_COUNT)]

# create squares
for row in range(ROW_COUNT):
    for col in range(COL_COUNT):

        # create square and add to circuit
        newSquare = Square(0, row, col)
        qc = newSquare.add_to_circuit(qc)

        # add square to organized data structures
        squares.append(newSquare)
        square_rows[row][newSquare.name] = newSquare
        square_cols[col][newSquare.name] = newSquare
        square_boxes[newSquare.box][newSquare.name] = newSquare

# create board
board = Board(starting_values, rows=3, cols=3, boxes=1)

# set board values on circuit
board.add_values(qc)
qc.draw('mpl')

In [ ]:

# add registers to circuit
conditions = QuantumRegister(CONDITION_COUNT, name="condition")
auxiliary = QuantumRegister(1, name="auxiliary")
qc.add_register(conditions)
qc.add_register(auxiliary)

# apply oracle on circuit
for iter in range(CIRCUIT_ITERATIONS):
    oracle(qc, squares, square_rows, square_cols, square_boxes, conditions, auxiliary)
    diffuser(qc, np.array(np.arange(len(squares) * VALUE_MAG))[board.diffuser_map].tolist())
    if not iter % 100:
        print(iter)

Now time to run it all

In [ ]:
# Simulate and plot results
aer_simulator = Aer.get_backend('aer_simulator')
transpiled_qc = transpile(qc, aer_simulator)
result = aer_simulator.run(transpiled_qc).result()
plot_histogram(result.get_counts())